In [67]:
import requests
import json
import time
from strava_api_details import * 
import os 
from typing import Dict, Tuple
from tqdm import tqdm
import urllib
import pandas as pd
import polyline
from geopy import distance
from math import sin, cos, sqrt, atan2, radians

id = 701809


def return_segment_api(id: int, access_token:str) -> None:
    os.system(f"curl -X GET https://www.strava.com/api/v3/segments/{id} -H 'Authorization: Bearer {access_token}' -o segment_{id}.JSON")


def return_segment_local(id, access_token) -> Dict:
    with open(f'segment_{id}.JSON') as f:
        json_data = json.load(f)
    return json_data


def get_params_from_segment_from_json(segment):
    return segment['name'], segment['average_grade'], segment['maximum_grade'], polyline.decode(segment['map']['polyline'])

def retrieve_segment(id: int, access_token:str):
    return_segment_api(id, access_token)
    seg_json = return_segment_local(id, access_token)
    name, grade, maximum_grade, lat_lon =get_params_from_segment_from_json(seg_json)
    print(f"fetching segment details for {name}")
    return name, grade, maximum_grade, lat_lon


def make_remote_request(url: str, params: dict):

   count = 1
   while True:
       try:
           response = requests.get((url + urllib.parse.urlencode(params)))
       except (OSError, urllib3.exceptions.ProtocolError) as error:
           print('\n')
           print('*' * 20, 'Error Occured', '*' * 20)
           print(f'Number of tries: {count}')
           print(f'URL: {url}')
           print(error)
           print('\n')
           count += 1
           continue
       break

   return response

def elevation_function(lat:float, lon:float):
   time.sleep(1)
   url = 'https://api.opentopodata.org/v1/eudem25m?'
   params = {'locations': f"{lat},{lon}"}
   result = make_remote_request(url, params)
   return result.json()['results'][0]['elevation']

def get_distance_delta(point1: list, point2: list) -> float:
    return distance.distance(point1, point2).km * 1000 # return delta in meters 

def get_elevation_delta(point1: list, point2:list) -> float:
    point1_elev = elevation_function(point1[0], point1[1])
    point2_elev = elevation_function(point2[0], point2[1])
    return point2_elev - point1_elev

def get_run_rise_grade(point1:list, point2:list) -> Tuple[float, float, float]:
    run  = get_distance_delta(point1, point2)
    rise = get_elevation_delta(point1, point2)
    return run, rise, rise/run *100 

In [59]:
name, grade, maximum_grade, lat_lon = retrieve_segment(id, access_token)

In [78]:
def get_seg_details_as_list():
    retrieve_segment(id, access_token)
    storage_list = []
    print('Remotely fetching elevation data ')
    for i in range(len(lat_lon)-1):
        storage_list.append(get_run_rise_grade(lat_lon[i],lat_lon[i+1]))




IndexError: list index out of range